In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

In [ ]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=05c73fd690a4c0a138f4c99adaa6286d772a7b34dc53ff9733e40e2e0a4b5340
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
import tqdm
import yfinance as yf
from torch.utils.data import DataLoader
from torch.distributions import Categorical
import tqdm
from google.colab import runtime
# Finance Data
import pandas as pd
import pandas_ta as ta
from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from torch.utils.data import DataLoader


seaborn.set_context(context="talk")
%matplotlib inline

In [ ]:
class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        pass

    def __repr__(self):
        return self.__class__.__name__ + "()"


class SecondOfMinute(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5


class MinuteOfHour(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5


class HourOfDay(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5


class DayOfWeek(TimeFeature):
    """Day of week encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5


class DayOfMonth(TimeFeature):
    """Day of month encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5


class DayOfYear(TimeFeature):
    """Day of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5


class MonthOfYear(TimeFeature):
    """Month of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5


class WeekOfYear(TimeFeature):
    """Week of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.isocalendar().week - 1) / 52.0 - 0.5


def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.
    Parameters
    ----------
    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
    """
    #print(f"freq_str: {freq_str}")

    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
        offsets.Second: [
            SecondOfMinute,
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
    }

    offset = to_offset(freq_str)
    #print(offset)

    for offset_type, feature_classes in features_by_offsets.items():
        #print(f"offset_type: {offset_type}")
        if isinstance(offset, offset_type):
            #print(cls for cls in feature_classes)
            return [cls() for cls in feature_classes]

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}
    The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
    """
    raise RuntimeError(supported_freq_msg)


def time_features(dates, freq='h'):
    #print(f"Entered time_features!!")
    #print(dates)
    return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)])


In [ ]:
import pandas as pd
import numpy as np

def datetime_to_one_hot(datetime_values):
    # Convert input to a DatetimeIndex if it's not already
    if isinstance(datetime_values, pd.Series):
        datetime_values = pd.to_datetime(datetime_values)
    elif isinstance(datetime_values, pd.Index):
        datetime_values = pd.DatetimeIndex(datetime_values)
    else:
        raise ValueError("Input should be a Pandas Series or DatetimeIndex.")

    # Extract the day of the week (0=Monday, 6=Sunday)
    day_of_week = datetime_values.dayofweek

    # Extract the day of the month (1-31)
    day_of_month = datetime_values.day

    # Extract the week of the month (1-5)
    week_of_month = (datetime_values.day - 1) // 7 + 1

    # Extract the month of the year (1-12)
    month_of_year = datetime_values.month

    # One-hot encode each component
    one_hot_day_of_week = np.eye(7)[day_of_week]
    one_hot_day_of_month = np.eye(31)[day_of_month - 1]  # day_of_month ranges from 1-31, so subtract 1 for 0-based indexing
    one_hot_week_of_month = np.eye(5)[week_of_month - 1]  # week_of_month ranges from 1-5, so subtract 1 for 0-based indexing
    one_hot_month_of_year = np.eye(12)[month_of_year - 1]  # month_of_year ranges from 1-12, so subtract 1 for 0-based indexing

    # Concatenate all one-hot vectors along the second axis
    one_hot_combined = np.concatenate([one_hot_day_of_week, one_hot_day_of_month, one_hot_week_of_month, one_hot_month_of_year], axis=1)

    return one_hot_combined


In [ ]:
def extract_data(start_year, end_year=2023, ticker="^SPX"):
    data = yf.Ticker(ticker).history(period="max")
    data = data.dropna()

    def add_features(data):
      # Assuming your DataFrame is named 'data'
      data.ta.sma(close="Close", length=50, append=True)
      data.ta.sma(close="Close", length=200, append=True)
      #data.ta.ichimoku(close="Close", append=True)
      #data.ta.macd(close="Close", append=True)
      data.ta.rsi(close="Close", append=True)
      data.ta.bbands(close="Close", append=True)
      data.ta.macd(close="Close", append=True)
      data.ta.ichimoku(close="Close", append=True)
      data.ta.smi(close="Close", append=True)
      data.ta.willr(close="Close", low="Low", high="High", append=True)
      data.ta.stoch(close="Close", low="Low", high="High", append=True)
      data.ta.fisher(low="Low", high="High", append=True)
      data.ta.atr(low="Low", high="High", close="Close", append=True)
      #data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
      data.ta.obv(volume="Volume", close="Close", append=True)
      data.ta.zscore(close="Close", append=True)
      data.ta.entropy(close="Close", append=True)
      return data

    data = add_features(data)
    ## Columns to Drop
    drop = ['Volume', 'Dividends', 'Stock Splits']
    data = data.drop(drop, axis=1)
    data = data.dropna()
    start_year = data.index[0].year if start_year is None else start_year
    data = data[data.index.year >= start_year-1] if start_year is not None else data
    data = data[data.index.year <= end_year] if end_year is not None else data
    #print(f"start year: {start_year}")
    #print(data)
    return data

df = extract_data(1990)
#df = df['Close']
df

,Open,High,Low,Close,SMA_50,SMA_200,RSI_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,...,SMIo_5_20_5,WILLR_14,STOCHk_14_3_3,STOCHd_14_3_3,FISHERT_9_1,FISHERTs_9_1,ATRr_14,OBV,ZS_30,ENTP_10
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,277.720001,277.720001,273.809998,275.309998,274.762200,268.111900,48.838065,274.613084,277.267993,279.922903,...,-0.057734,-77.409688,54.631409,58.669202,0.388937,0.797978,2.233492,2.680319e+10,0.229301,3.325722
1989-01-04 00:00:00-05:00,275.309998,279.750000,275.309998,279.429993,274.677599,268.165350,58.288271,274.697225,277.787994,280.878764,...,-0.011009,-15.361706,54.945716,57.522858,0.248036,0.388937,2.391100,2.695289e+10,1.197195,3.325502
1989-01-05 00:00:00-05:00,279.429993,281.510010,279.429993,280.010010,274.632200,268.221200,59.424554,274.949366,278.373999,281.798632,...,0.014795,-19.480489,62.582706,57.386610,0.507648,0.248036,2.368880,2.712693e+10,1.312146,3.326466
1989-01-06 00:00:00-05:00,280.010010,282.059998,280.010010,280.670013,274.598000,268.280000,60.735306,274.775049,278.628003,282.480957,...,0.030888,-16.848292,82.769838,66.766086,0.908969,0.507648,2.346102,2.728826e+10,1.453896,3.327660
1989-01-09 00:00:00-05:00,280.670013,281.890015,280.320007,280.980011,274.590000,268.368150,61.366579,275.167730,279.280005,283.392280,...,0.037330,-13.090746,83.526824,76.293122,1.352505,0.908969,2.290666,2.745144e+10,1.477710,3.329150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,4753.919922,4772.939941,4736.770020,4754.629883,4468.218584,4335.861046,71.069402,4694.534676,4741.732031,4788.929386,...,-0.054860,-10.098875,80.663261,83.908053,2.832599,2.986470,41.184150,1.379242e+12,1.638294,3.350524
2023-12-26 00:00:00-05:00,4758.859863,4784.720215,4758.450195,4774.750000,4477.157988,4340.426846,72.704374,4694.610700,4748.570020,4802.529339,...,-0.035086,-4.185293,90.737754,85.128169,2.967104,2.832599,40.391734,1.381756e+12,1.736890,3.350531
2023-12-27 00:00:00-05:00,4773.450195,4785.390137,4768.899902,4781.580078,4485.316992,4345.055946,73.256889,4692.545294,4751.212012,4809.878729,...,-0.019366,-1.730508,94.661774,88.687597,3.241961,2.967104,38.684484,1.384504e+12,1.720615,3.349759


In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class StockDataset(Dataset):
    def __init__(self, tickers, flag='train', size=None, one_hot_datetime=False,
                 features='S', target='Close', scale=True, timeenc=0, freq='d', batch_size=5,
                 data_start_year=1990, data_end_year=2023):
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]

        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.data_start_year = data_start_year
        self.data_end_year = data_end_year

        self.features = features
        self.target = target
        self.scale = scale
        self.one_hot_datetime = one_hot_datetime
        self.timeenc = timeenc
        self.freq = freq
        self.scaler = StandardScaler()
        self.tickers = tickers.split()
        self.ticker_database = {}
        self.batch_size = batch_size
        self.min_year = 0
        self.__read_data__()

    def __len__(self):
        return len(self.years) - 1

    def get_min_year(self):
        min_year = 0
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=None, ticker=ticker)
            min_year = max(min_year, self.ticker_database[ticker].index.year.min())
        return min_year + 2

    def __read_data__(self):
        print(f"Loading following tickers: {self.tickers}\n")

        self.min_year = max(self.get_min_year(), self.data_start_year)
        print(f"Dataset Start Year: {self.min_year} | End Year: {self.data_end_year}")
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=self.min_year, end_year=self.data_end_year, ticker=ticker)
            self.ticker_database[ticker]['year'] = self.ticker_database[ticker].index.year

        self.years = self.ticker_database[self.tickers[0]]['year'].unique()
        print(f"years: {self.years}")
        self.data_by_year = {year: {ticker: self.ticker_database[ticker][self.ticker_database[ticker]['year'] == year] for ticker in self.tickers} for year in self.years}

        self.data_len = len(self.years)
        print(f"Data Normalization: {self.scale}")
        print(f"DateTime is one-hot: {self.one_hot_datetime}")

    def __getitem__(self, idx):
        """
        Final item form is a list containing [seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates]
        batches_x = batches[0]
        batches_y = batches[1]
        batches_x_mark = batches[2]
        batches_y_mark = batches[3]
        batches_x_dates = batches[4]
        batches_y_dates = batches[5]

        Each batches, contain batch data of size (batch_size, seq_len, num_features)
        For example, if seq_len = 24, batch_size = 5, num_features = 32,
        Each item of batches_x is a tensor of (5, 24, 32)
        """
        year = self.min_year + idx if idx >= 0 else self.max_year + idx + 1
        #print(f"Stock Dataset Yeaer: {year}")
        raw_datas = []

        for ticker in self.tickers:
            if year - 1 in self.data_by_year:
                prev_year_data = self.data_by_year[year - 1][ticker].tail(self.seq_len + self.pred_len - 1)
                #print(f"Prev year data for {ticker} in {year}: {prev_year_data.index}")
            else:
                print(f"Previous Year Data is Insufficient, Year: {year-1}, Ticker: {ticker}")
                prev_year_data = pd.DataFrame()

            ticker_data = pd.concat([prev_year_data, self.data_by_year[year][ticker]])
            #print(f"Ticker data for {ticker} in {year}: {ticker_data.index}")
            ticker_data['date'] = ticker_data.index
            raw_datas.append(ticker_data)
            """print(f"Ticker: {ticker}")
            print(len(ticker_data))
            print(ticker_data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        seq_x_dates = []
        seq_y_dates = []

        for item in raw_datas:
            x, y, x_mark, y_mark, x_dates, y_dates = self.make_data(item)
            seq_x.append(x)
            seq_y.append(y)
            seq_x_mark.append(x_mark)
            seq_y_mark.append(y_mark)
            seq_x_dates.append(x_dates)
            seq_y_dates.append(y_dates)

        # Combine all tickers into a single batch and slice them into mini-batches
        #print(len(seq_x))
        return self.create_batches(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates), year

    def make_data(self, raw_data):
        cols = list(raw_data.columns)
        cols.remove(self.target)
        cols.remove('date')
        cols.remove('year')
        raw_data = raw_data[['date'] + cols + [self.target]]

        if self.features == 'MS':
            cols_data = raw_data.columns[1:]
            data = raw_data[cols_data]
        elif self.features == 'S':
            data = raw_data['Close']

        if self.scale:
            data = self.scaler.fit_transform(data.values)

        if self.one_hot_datetime:
          #print(type(pd.to_datetime(raw_data['date'].values)))
          data_stamp = datetime_to_one_hot(pd.to_datetime(raw_data['date'].values))
          #print(one_hot_data_stamp.shape)
        else:
          data_stamp = time_features(pd.to_datetime(raw_data['date'].values), freq=self.freq)
          data_stamp = data_stamp.transpose(1, 0)

        """print("----Yearly data----")
        print(data[0].shape)
        print(len(data))
        print(data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        x_dates = []
        y_dates = []

        for i in range(len(data) - self.pred_len - self.seq_len + 1):
            s_begin = i
            s_end = s_begin + self.seq_len
            r_begin = s_end - self.label_len
            r_end = r_begin + self.label_len + self.pred_len

            seq_x.append(data[s_begin:s_end])
            seq_y.append(data[r_begin:r_end])
            seq_x_mark.append(data_stamp[s_begin:s_end])
            seq_y_mark.append(data_stamp[r_begin:r_end])
            x_dates.append(raw_data['date'][s_begin:s_end])
            y_dates.append(raw_data['date'][r_begin:r_end])
            """print(f"data[s_begin:s_end]: {data[s_begin:s_end]}")
            print(f"data[r_begin:r_end]: {data[r_begin:r_end]}")
            print(f"raw_data['date'][s_begin:s_end]: {raw_data['date'][s_begin:s_end]}")
            print(f"raw_data['date'][r_begin:r_end]: {raw_data['date'][r_begin:r_end]}")
            print("=======================================")"""

        return torch.tensor(np.array(seq_x)), torch.tensor(np.array(seq_y)), torch.tensor(np.array(seq_x_mark)), torch.tensor(np.array(seq_y_mark)), x_dates, y_dates

    def create_batches(self, seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates, dates=None):
        batches_x = []
        batches_y = []
        batches_x_mark = []
        batches_y_mark = []
        batches_x_dates = []
        batches_y_dates = []
        batch_size = self.batch_size
        #print(dates[-1])
        for x, y, x_mark, y_mark, x_dates, y_dates in zip(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates):
            #print(x.shape)
            for i in range(0, x.shape[0], batch_size):
                batches_x.append(x[i:i + batch_size])
                batches_y.append(y[i:i + batch_size])
                #print(f"x[i:i + batch_size]: {x[i:i + batch_size].shape}")
                #print(f"y[i:i + batch_size]: {y[i:i + batch_size].shape}")
                batches_x_mark.append(x_mark[i:i + batch_size])
                batches_y_mark.append(y_mark[i:i + batch_size])
                batches_x_dates.append(x_dates[i:i + batch_size])
                batches_y_dates.append(y_dates[i:i + batch_size])
                """print(f"x[i:i + batch_size]: {x[i:i + batch_size]}")
                print(f"y[i:i + batch_size]: {y[i:i + batch_size]}")
                print(f"x_dates[i:i + batch_size]: {x_dates[i:i + batch_size]}")
                print(f"y_dates[i:i + batch_size]: {y_dates[i:i + batch_size]}")
                print(f"------------------------------------------------------")"""

        return batches_x, batches_y, batches_x_mark, batches_y_mark, batches_x_dates, batches_y_dates

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [ ]:
dataset = StockDataset(tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS')

Loading following tickers: ['^SPX']



ERROR:yfinance:$^SPX: possibly delisted; no price data found  (1d 1925-09-23 -> 2024-08-29)


IndexError: iloc cannot enlarge its target object

In [ ]:
batches, year = dataset[0]

# Model

## CARD

In [ ]:
!pip install einops

In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from einops import rearrange
import numpy as np

class Transpose(nn.Module):
    def __init__(self, *dims, contiguous=False):
        super().__init__()
        self.dims, self.contiguous = dims, contiguous
    def forward(self, x):
        if self.contiguous: return x.transpose(*self.dims).contiguous()
        else: return x.transpose(*self.dims)

class Model(nn.Module):
    def __init__(self, config, **kwargs):

        super().__init__()
        self.model = CARDformer(config)
        self.task_name = config.task_name
        self.time_embedding = nn.Linear(config.datetime_features, config.enc_in)
        self.config = config

    def forward(self, x, batch_x_mark, *args, **kwargs):
        if self.config.one_hot_datetime:
            x = x + self.time_embedding(batch_x_mark)
        x = x.permute(0,2,1)
        mask = args[-1]
        x = self.model(x, batch_x_mark, mask = mask)
        if self.task_name != 'classification':
            x = x.permute(0,2,1)
        return x

class CARDformer(nn.Module):
    def __init__(self,
                 config,**kwargs):
        super().__init__()
        self.config = config
        self.patch_len  = config.patch_len
        self.stride = config.stride
        self.d_model = config.d_model
        self.task_name = config.task_name
        patch_num = int((config.seq_len - self.patch_len)/self.stride + 1)
        self.patch_num = patch_num
        self.W_pos_embed = nn.Parameter(torch.randn(patch_num,config.d_model)*1e-2)
        self.model_token_number = 0

        if self.model_token_number > 0:
            self.model_token = nn.Parameter(torch.randn(config.enc_in,self.model_token_number,config.d_model)*1e-2)

        self.total_token_number = (self.patch_num  + self.model_token_number + 1)
        config.total_token_number = self.total_token_number

        self.W_input_projection = nn.Linear(self.patch_len, config.d_model)
        self.input_dropout  = nn.Dropout(config.dropout)

        self.use_statistic = config.use_statistic
        self.W_statistic = nn.Linear(2,config.d_model)
        self.cls = nn.Parameter(torch.randn(1,config.d_model)*1e-2)

        if config.task_name == 'long_term_forecast' or config.task_name == 'short_term_forecast':
            self.W_out = nn.Linear((patch_num+1+self.model_token_number)*config.d_model, config.pred_len)
        elif config.task_name == 'imputation' or config.task_name == 'anomaly_detection':
            self.W_out = nn.Linear((patch_num+1+self.model_token_number)*config.d_model, config.seq_len)
        elif config.task_name == 'classification':
            self.W_out = nn.Linear(config.d_model*config.enc_in, config.num_class)

        self.Attentions_over_token = nn.ModuleList([Attenion(config) for i in range(config.e_layers)])
        self.Attentions_over_channel = nn.ModuleList([Attenion(config,over_hidden = True) for i in range(config.e_layers)])
        self.Attentions_mlp = nn.ModuleList([nn.Linear(config.d_model,config.d_model)  for i in range(config.e_layers)])
        self.Attentions_dropout = nn.ModuleList([nn.Dropout(config.dropout)  for i in range(config.e_layers)])
        self.Attentions_norm = nn.ModuleList([nn.Sequential(Transpose(1,2), nn.BatchNorm1d(config.d_model,momentum = config.momentum), Transpose(1,2)) for i in range(config.e_layers)])

    def forward(self, z, batch_x_mark, *args, **kwargs):
        b,c,s = z.shape
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast' or self.task_name == 'anomaly_detection':
            z_mean = torch.mean(z,dim = (-1),keepdims = True)
            z_std = torch.std(z,dim = (-1),keepdims = True)
            z =  (z - z_mean)/(z_std + 1e-4)

        elif self.task_name == 'imputation':
            mask = kwargs['mask'].permute(0,2,1)
            z_mean = torch.sum(z, dim=-1) / torch.sum(mask == 1, dim=-1)
            z_mean = z_mean.unsqueeze(-1)
            z = z - z_mean
            z = z.masked_fill(mask == 0, 0)
            z_std = torch.sqrt(torch.sum(z * z, dim=-1) /
                           torch.sum(mask == 1, dim=-1) + 1e-5)
            z_std = z_std.unsqueeze(-1)
            z /= z_std + 1e-4

        if self.config.one_hot_datetime:
          zcube = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)
          z_embed = self.input_dropout(self.W_input_projection(zcube))
        else:
          zcube = z.unfold(dimension=-1, size=self.patch_len, step=self.stride)
          z_embed = self.input_dropout(self.W_input_projection(zcube))+ self.W_pos_embed

        if self.use_statistic:
            z_stat = torch.cat((z_mean,z_std),dim = -1)
            if z_stat.shape[-2]>1:
                z_stat = (z_stat - torch.mean(z_stat,dim =-2,keepdims = True))/( torch.std(z_stat,dim =-2,keepdims = True)+1e-4)
            z_stat = self.W_statistic(z_stat)
            z_embed = torch.cat((z_stat.unsqueeze(-2),z_embed),dim = -2)
        else:
            cls_token = self.cls.repeat(z_embed.shape[0],z_embed.shape[1],1,1)
            z_embed = torch.cat((cls_token,z_embed),dim = -2)

        inputs = z_embed
        b,c,t,h = inputs.shape
        for a_2,a_1,mlp,drop,norm  in zip(self.Attentions_over_token, self.Attentions_over_channel,self.Attentions_mlp ,self.Attentions_dropout,self.Attentions_norm ):
            output_1 = a_1(inputs.permute(0,2,1,3)).permute(0,2,1,3)
            output_2 = a_2(output_1)
            outputs = drop(mlp(output_1+output_2))+inputs
            outputs = norm(outputs.reshape(b*c,t,-1)).reshape(b,c,t,-1)
            inputs = outputs

        if self.task_name != 'classification':
            z_out = self.W_out(outputs.reshape(b,c,-1))
            z = z_out *(z_std+1e-4)  + z_mean
        else:
            z = self.W_out(torch.mean(outputs[:,:,:,:],dim = -2).reshape(b,-1))
        return z


class Attenion(nn.Module):
    def __init__(self,config, over_hidden = False,trianable_smooth = False,untoken = False, *args, **kwargs):
        super().__init__()


        self.over_hidden = over_hidden
        self.untoken = untoken
        self.n_heads = config.n_heads
        self.c_in = config.enc_in
        self.qkv = nn.Linear(config.d_model, config.d_model * 3, bias=True)



        self.attn_dropout = nn.Dropout(config.dropout)
        self.head_dim = config.d_model // config.n_heads


        self.dropout_mlp = nn.Dropout(config.dropout)
        self.mlp = nn.Linear( config.d_model,  config.d_model)



        self.norm_post1  = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(config.d_model,momentum = config.momentum), Transpose(1,2))
        self.norm_post2  = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(config.d_model,momentum = config.momentum), Transpose(1,2))

        self.norm_attn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(config.d_model,momentum = config.momentum), Transpose(1,2))


        self.dp_rank = config.dp_rank
        self.dp_k = nn.Linear(self.head_dim, self.dp_rank)
        self.dp_v = nn.Linear(self.head_dim, self.dp_rank)


        self.ff_1 = nn.Sequential(nn.Linear(config.d_model, config.d_ff, bias=True),
                        nn.GELU(),
                        nn.Dropout(config.dropout),
                        nn.Linear(config.d_ff, config.d_model, bias=True)
                       )

        self.ff_2= nn.Sequential(nn.Linear(config.d_model, config.d_ff, bias=True),
                        nn.GELU(),
                        nn.Dropout(config.dropout),
                        nn.Linear(config.d_ff, config.d_model, bias=True)
                                )
        self.merge_size = config.merge_size

        ema_size = max(config.enc_in,config.total_token_number,config.dp_rank)
        ema_matrix = torch.zeros((ema_size,ema_size))
        alpha = config.alpha
        ema_matrix[0][0] = 1
        for i in range(1,config.total_token_number):
            for j in range(i):
                ema_matrix[i][j] =  ema_matrix[i-1][j]*(1-alpha)
            ema_matrix[i][i] = alpha
        self.register_buffer('ema_matrix',ema_matrix)




    def ema(self,src):
        return torch.einsum('bnhad,ga ->bnhgd',src,self.ema_matrix[:src.shape[-2],:src.shape[-2]])


    def ema_trianable(self,src):
        alpha = F.sigmoid(self.alpha)

        weights = alpha * (1 - alpha) ** self.arange[-src.shape[-2]:]


        w_f = torch.fft.rfft(weights,n = src.shape[-2]*2)
        src_f = torch.fft.rfft(src.float(),dim = -2,n = src.shape[-2]*2)
        src_f = (src_f.permute(0,1,2,4,3)*w_f)
        src1 =torch.fft.irfft(src_f.float(),dim = -1,n=src.shape[-2]*2)[...,:src.shape[-2]].permute(0,1,2,4,3)#.half()
        return src1



    def dynamic_projection(self,src,mlp):
        src_dp = mlp(src)
        src_dp = F.softmax(src_dp,dim = -1)
        src_dp = torch.einsum('bnhef,bnhec -> bnhcf',src,src_dp)
        return src_dp




    def forward(self, src, *args,**kwargs):
        B,nvars, H, C, = src.shape
        qkv = self.qkv(src).reshape(B,nvars, H, 3, self.n_heads, C // self.n_heads).permute(3, 0, 1,4, 2, 5)
        q, k, v = qkv[0], qkv[1], qkv[2]

        if not self.over_hidden:
            attn_score_along_token = torch.einsum('bnhed,bnhfd->bnhef', self.ema(q), self.ema(k))/ self.head_dim ** -0.5
            attn_along_token = self.attn_dropout(F.softmax(attn_score_along_token, dim=-1) )
            output_along_token = torch.einsum('bnhef,bnhfd->bnhed', attn_along_token, v)
        else:
            v_dp,k_dp = self.dynamic_projection(v,self.dp_v) , self.dynamic_projection(k,self.dp_k)
            attn_score_along_token = torch.einsum('bnhed,bnhfd->bnhef', self.ema(q), self.ema(k_dp))/ self.head_dim ** -0.5
            attn_along_token = self.attn_dropout(F.softmax(attn_score_along_token, dim=-1) )
            output_along_token = torch.einsum('bnhef,bnhfd->bnhed', attn_along_token, v_dp)

        attn_score_along_hidden = torch.einsum('bnhae,bnhaf->bnhef', q,k)/ q.shape[-2] ** -0.5
        attn_along_hidden = self.attn_dropout(F.softmax(attn_score_along_hidden, dim=-1) )
        output_along_hidden = torch.einsum('bnhef,bnhaf->bnhae', attn_along_hidden, v)
        merge_size = self.merge_size
        if not self.untoken:
            output1 = rearrange(output_along_token.reshape(B*nvars,-1,self.head_dim),
                            'bn (hl1 hl2 hl3) d -> bn  hl2 (hl3 hl1) d',
                            hl1 = self.n_heads//merge_size, hl2 = output_along_token.shape[-2] ,hl3 = merge_size
                            ).reshape(B*nvars,-1,self.head_dim*self.n_heads)


            output2 = rearrange(output_along_hidden.reshape(B*nvars,-1,self.head_dim),
                            'bn (hl1 hl2 hl3) d -> bn  hl2 (hl3 hl1) d',
                            hl1 = self.n_heads//merge_size, hl2 = output_along_token.shape[-2] ,hl3 = merge_size
                            ).reshape(B*nvars,-1,self.head_dim*self.n_heads)

        output1 = self.norm_post1(output1)
        output1 = output1.reshape(B,nvars, -1, self.n_heads * self.head_dim)
        output2 = self.norm_post2(output2)
        output2 = output2.reshape(B,nvars, -1, self.n_heads * self.head_dim)
        src2 =  self.ff_1(output1)+self.ff_2(output2)

        src = src + src2
        src = src.reshape(B*nvars, -1, self.n_heads * self.head_dim)
        src = self.norm_attn(src)

        src = src.reshape(B,nvars, -1, self.n_heads * self.head_dim)
        return src

In [ ]:
import torch

def evaluate_direction_accuracy(output, batch_y):
    """
    Evaluate the accuracy of direction predictions and compare it to a baseline model.

    Parameters:
    output (torch.Tensor): Tensor of predicted values with shape (batch_size, 96, 1).
    batch_y (torch.Tensor): Tensor of true values with shape (batch_size, 96, 1).

    Returns:
    correct_predictions (int): Number of times the predicted direction was correct.
    accuracy (float): Fraction of correct predictions out of total comparisons.
    baseline_correct (int): Number of times the baseline prediction was correct.
    baseline_accuracy (float): Fraction of correct predictions by the baseline out of total comparisons.
    """
    # Compute the direction of change for each tensor
    def compute_direction(tensor):
        # Compute the difference between consecutive elements
        diff = tensor[:, 1:, :] - tensor[:, :-1, :]
        # Convert differences to binary direction indicators: 1 for increase, -1 for decrease
        direction = torch.sign(diff)
        return direction

    # Get direction indicators for both tensors
    #print(batch_y[:,0,:].shape)
    output = torch.cat((batch_y[:, 0, :].unsqueeze(2), output), dim=1)
    #print(output)
    output_direction = compute_direction(output)
    batch_y_direction = compute_direction(batch_y)
    #print(f"output_direction: {output_direction.shape} batch_y_direction: {batch_y_direction.shape}")
    # Compare the directions
    correct_predictions = (output_direction == batch_y_direction).sum().item()
    #print(f"correct_predictions: {correct_predictions}")
    total_comparisons = output_direction.numel()
    #print(f"total_comparisons: {total_comparisons}")
    accuracy = correct_predictions / total_comparisons

    # Compute baseline predictions (always predict increase)
    baseline_direction = torch.ones_like(output_direction)  # Baseline always predicts increase
    baseline_correct = (baseline_direction == batch_y_direction).sum().item()
    baseline_accuracy = baseline_correct / total_comparisons

    return correct_predictions, baseline_correct, total_comparisons

# Example usage (replace these with your actual tensors)
output = torch.randn(10, 1, 1)  # Example output tensor
batch_y = torch.randn(10, 2, 1)  # Example batch_y tensor

correct_predictions, baseline_correct, total_comparisons = evaluate_direction_accuracy(output, batch_y)

# Prediction Test

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'short_term_forecast'
    num_class: int =2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = True
    datetime_features: int = 55

    ## Training
    run_name: str = "one_hot_test"
    validation_years: int = 1
    test_years: int = 1
    ticker: str = "^SPX"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [ ]:
configs = Args()
card = Model(configs)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_optim = torch.optim.Adam(card.parameters(), lr=configs.learning_rate)
loss_fn = nn.MSELoss()
dataset = StockDataset(tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS', batch_size=5, one_hot_datetime=True)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
DateTime is one-hot: True


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
range_limit = configs.rolling_window+configs.validation_years+configs.test_years
loop_range = tqdm.tqdm(range(len(dataset)-range_limit))
card = card.to(device)
for iteration in loop_range:
  for window_iteration in range(configs.window_epoch):
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    starting_idx = iteration // 10
    for i in range((configs.rolling_window+configs.validation_years+configs.test_years)):
        validation, test = False, False
        if i == configs.rolling_window:
          validation = True
        elif i == (configs.rolling_window+1):
          test = True
        else:
          batches, year  = dataset[i+iteration]
          batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
          for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            true_batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
            dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
            #print(batch_x.shape)

            outputs = card(batch_x, batch_x_mark, dec_inp, None)
            print(f"batches_y shape: {batch_y.shape}")
            print(f"outputs shape: {outputs.shape}")

            f_dim = -1 if configs.features == 'MS' else 0

            batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(device)
            print(f"batch_y_direction: {batch_y_direction}")
            batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)
            print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape} | batch_y_direction: {batch_y_direction.shape}")
            loss = loss_fn(outputs, batch_y)
            epoch_loss.append(loss.item())
            loss.backward()
            model_optim.step()

            #print(f"outputs shape: {outputs.shape} | true_batch_y[:,-97:,:]: {true_batch_y[:,-97:,-1:].shape}")
            correct_predictions, baseline_correct, total_comparisons = evaluate_direction_accuracy(outputs, true_batch_y[:,-(configs.pred_len+1):,-1:])
            total_hits += correct_predictions
            total_data +=total_comparisons
            total_ol += baseline_correct
        print(f"Year: {year} total_comparisons: {total_data} training_hit_ratio: {total_hits/total_data} | ol_hit_ratio: {total_ol/total_data}")
        break
    break
  break

  0%|          | 0/22 [00:00<?, ?it/s]

batches_y shape: torch.Size([5, 19, 32])
outputs shape: torch.Size([5, 1, 32])
batch_y_direction: tensor([[[1.0219],
         [1.3905]],

        [[1.3905],
         [1.3360]],

        [[1.3360],
         [1.1549]],

        [[1.1549],
         [0.9516]],

        [[0.9516],
         [1.0447]]], device='cuda:0')
outputs: torch.Size([5, 1, 32]) | batch_Y: torch.Size([5, 1, 1]) | batch_y_direction: torch.Size([5, 2, 1])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([5, 1, 1])) that is different to the input size (torch.Size([5, 1, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.

# Trainer

In [62]:
class Trainer():
  def __init__(self, configs):
    self.configs = configs

    ticker_str = configs.tickers.replace(" ", "_")
    self.run_name = f"[{ticker_str}]_valYrs:{self.configs.validation_years}_testYrs{self.configs.test_years}_reset:{self.configs.reset_model}_{configs.run_name}"
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.set_seed(configs.seed)
    self.dataset = self.make_data()
    self.model = Model(configs)
    self.model.to(self.device)
    self.loss_fn = nn.MSELoss()
    self.model_optim = torch.optim.Adam(self.model.parameters(), lr=configs.learning_rate)
    self.writer = SummaryWriter(f"/content/drive/MyDrive/code/fintransformer/runs/{self.run_name}")
    path = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}"
    if not os.path.exists(path):
      os.mkdir(path)
      print(f"Save File Directory Made at {path}\n")
    else:
      print(f"Directory Already Exists at {path}")

  def set_seed(self, seed):
      torch.manual_seed(seed)
      torch.cuda.manual_seed(seed)
      #torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
      np.random.seed(seed)
      random.seed(seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

  def make_data(self):
    def collate_fn(batch):
      seq_x, seq_y, seq_x_mark, seq_y_mark, year = batch[0]
      return seq_x, seq_y, seq_x_mark, seq_y_mark, year
    dataset = StockDataset(tickers=self.configs.tickers, timeenc=1, freq='d', size=[self.configs.seq_len, self.configs.label_len, self.configs.pred_len],
                           features=self.configs.features, batch_size=self.configs.batch_size, one_hot_datetime=self.configs.one_hot_datetime,
                           data_start_year=self.configs.data_start_year, data_end_year=self.configs.data_end_year)
    """prices, labels, year = next(iter(val_loader))
    feature_num = prices.shape[2]
    label_num = labels.shape[1]
    print(f"Data shape: {prices.shape} | Num Features: {feature_num}")"""
    return dataset

  def run(self):
    ### Training ###
    print(f"Training model: {self.configs.run_name}")
    print()
    range_limit = self.configs.rolling_window + self.configs.validation_years + self.configs.test_years
    loop_range = tqdm.tqdm(range(len(self.dataset)-range_limit))
    recent_save_path = ""
    val_acc_arr = []
    test_acc_arr = []
    for iteration in loop_range:
      highest_acc = 0.0
      highest_test_acc = 0.0
      if self.configs.reset_model and iteration != 0:
        self.model = Model(self.configs)
        self.model.to(self.device)
        self.model_optim = torch.optim.Adam(self.model.parameters(), lr=self.configs.learning_rate)
        print(f"Model Has Been Reset to Random Parameters")
      else:
        self.load_model(recent_save_path) if recent_save_path != "" else None
      for window_iteration in range(self.configs.window_epoch):
        train_loss, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year = self.train(iteration, window_iteration, highest_acc)
        if average_val_accuracy > highest_acc:
            highest_acc = average_val_accuracy
            highest_test_acc = average_test_accuracy
            recent_save_path = self.save_model(average_val_accuracy, average_test_accuracy, window_iteration, validation_year, test_year)
      #self.writer.add_scalar("Train/Test Loss", train_loss, iteration)
      val_acc_arr.append([highest_acc, validation_year])
      test_acc_arr.append([highest_test_acc, test_year])
      print(f"***********************************************************")
      print(f"Highest Validation Accuracy in {validation_year[0]}~{validation_year[-1]}: {highest_acc*100:.2f}% | Highest Test Accuracy in {test_year}: {average_test_accuracy*100:.2f}%")
      print(f"***********************************************************")
      print("=============================New Training Set====================================")
    self.writer.flush()
    self.writer.close()
    #print(f"Highest Accuracy in Validation Set: {highest_acc*100:.2f}")
    print()
    for val, test in zip(val_acc_arr, test_acc_arr):
      print(f"Validation Accuracy in {val[1][0]}~{val[1][-1]} : {val[0]*100:.2f}% | Test Accuracy in {test[1]}: {test[0]*100:.2f}%")

    average_first_elements_val = sum(item[0] for item in val_acc_arr) / len(val_acc_arr)
    average_first_elements_test = sum(item[0] for item in test_acc_arr) / len(test_acc_arr)
    print(f"Average Validation Accuracy: {average_first_elements_val*100:.2f}% | Average Test Accuracy: {average_first_elements_test*100:.2f}%")
    return

  def train(self, iteration, window_iteration, highest_acc):
    epoch_loss = []
    val_epoch_loss = []
    training_start_year, training_end_year = 0, 0
    training_total_hits = 0
    training_total_data = 0
    training_total_ol = 0
    validation_year, test_year = [], 0
    validation_total_hits = 0
    validation_total_data = 0
    validation_total_ol = 0
    test_total_hits = 0
    test_total_data = 0
    test_total_ol = 0
    for i in range((self.configs.rolling_window+self.configs.validation_years+self.configs.test_years)):
      validation, test = False, False
      if i >= self.configs.rolling_window and i <= (self.configs.rolling_window+self.configs.validation_years-1):
        batches, year = self.dataset[i+iteration]
        #print(f"validation year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long, val_loss_epoch, total_hits, total_data, total_ol = self.eval_once(batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc=0, train_acc=0)
        validation_total_hits += total_hits
        validation_total_data += total_data
        validation_total_ol += total_ol
        val_epoch_loss.append(val_loss_epoch)
        validation_year.append(year)
        validation = True
        print(f"Year: {year} | Validation Accuracy: {(total_hits/total_data)*100:.2f}% | Only Long Accuracy: {(total_ol/total_data)*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
      elif i >= (self.configs.rolling_window+self.configs.validation_years):
        batches, year = self.dataset[i+iteration]
        #print(f"test year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long,  total_hits, total_data, total_ol = self.test(batches_x, batches_y, batches_x_mark, batches_y_mark, year)
        test_total_hits += total_hits
        test_total_data += total_data
        test_total_ol += total_ol
        test_year = year
        test = True
      else:
        self.model.train()
        batches, year = self.dataset[i+iteration]
        training_start_year = year if i == 0 else training_start_year
        training_end_year = year if i == self.configs.rolling_window-1 else training_end_year
        #print(f"train year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        for j, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
          self.model_optim.zero_grad()

          batch_x = batch_x.float().to(self.device)
          batch_y = batch_y.float().to(self.device)
          true_batch_y = batch_y.float().to(self.device)
          batch_x_mark = batch_x_mark.float().to(self.device)
          batch_y_mark = batch_y_mark.float().to(self.device)
          #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
          dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
          dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
          #print(batch_x.shape)

          outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

          f_dim = -1 if self.configs.features == 'MS' else 0
          #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
          batch_y_direction = batch_y[:, -(self.configs.pred_len+1):, f_dim:].to(self.device)
          outputs = outputs[:, -self.configs.pred_len:, f_dim:]
          batch_y = batch_y[:, -self.configs.pred_len:, f_dim:].to(self.device)

          loss = self.loss_fn(outputs, batch_y)
          epoch_loss.append(loss.item())
          loss.backward()
          self.model_optim.step()
        #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

          hit_count, only_long, count = evaluate_direction_accuracy(outputs, true_batch_y[:,-(self.configs.pred_len+1):,-1:])
          training_total_hits += hit_count
          training_total_data += count
          training_total_ol += only_long

    loss_epoch = np.mean(epoch_loss)
    average_val_accuracy = validation_total_hits / validation_total_data
    average_val_only_long = validation_total_ol / validation_total_data
    average_val_loss = np.mean(val_epoch_loss)
    average_test_accuracy = test_total_hits / test_total_data
    average_test_only_long = test_total_ol / test_total_data
    train_accuracy = training_total_hits / training_total_data
    train_only_long = training_total_ol / training_total_data
    print(f"Training Years: {training_start_year}~{training_end_year} | Training Accuracy: {train_accuracy*100:.2f}% | Training OL Accuracy: {train_only_long*100:.2f}%")
    print(f"Year: {validation_year[0]}~{validation_year[-1]} Validation Accuracy: {average_val_accuracy*100:.2f}% | Only Long Accuracy: {average_val_only_long*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
    print(f"Year: {test_year} | Test Accuracy: {average_test_accuracy*100:.2f}% | Only Long Accuracy: {average_test_only_long*100:.2f}%")
    print(f"------------------------------------------------")
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Accuracy", train_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Loss", loss_epoch, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Loss", average_val_loss, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Accuracy", average_val_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Test Accuracy", average_test_accuracy, window_iteration)
      #print(f"Year: {year} training_hit_ratio: {hit_count/batch_x.shape[0]} | ol_hit_ratio: {only_long/batch_x.shape[0]}")

    #print(f"train accuracy: {accuracy*100:.2f}% | only_long: {only_long*100:.2f}%")
    return loss_epoch, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year

  def eval_once(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc, train_acc, last=False):
    self.model.eval()
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        true_batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
        dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
        #print(batch_x.shape)

        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if self.configs.features == 'MS' else 0
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
        batch_y_direction = batch_y[:, -(self.configs.pred_len+1):, f_dim:].to(self.device)
        outputs = outputs[:, -self.configs.pred_len:, f_dim:]
        batch_y = batch_y[:, -self.configs.pred_len:, f_dim:].to(self.device)

        loss = self.loss_fn(outputs, batch_y)
        epoch_loss.append(loss.item())

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        hit_count, only_long, count = evaluate_direction_accuracy(outputs, true_batch_y[:,-(self.configs.pred_len+1):,-1:])
        total_hits += hit_count
        total_data += count
        total_ol += only_long
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    loss_epoch = np.mean(epoch_loss)
    #print(f"validation accuracy: {accuracy*100:.2f}% | val highest_acc: {highest_acc*100:.2f}% | val only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, loss_epoch, total_hits, total_data, total_ol

  def test(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year):
    self.model.eval()
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        true_batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
        dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
        #print(batch_x.shape)

        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if self.configs.features == 'MS' else 0
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
        batch_y_direction = batch_y[:, -(self.configs.pred_len+1):, f_dim:].to(self.device)
        outputs = outputs[:, -self.configs.pred_len:, f_dim:]
        batch_y = batch_y[:, -self.configs.pred_len:, f_dim:].to(self.device)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        hit_count, only_long, count = evaluate_direction_accuracy(outputs, true_batch_y[:,-(self.configs.pred_len+1):,-1:])
        total_hits += hit_count
        total_data += count
        total_ol += only_long
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    #print(f"test accuracy: {accuracy*100:.2f}% | test only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, total_hits, total_data, total_ol

  def save_model(self, val_acc, test_acc, epoch, validation_year, test_year):
    PATH = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}/reset:{self.configs.reset_model}_valYear:{validation_year}_testYear:{test_year}.pt"
    torch.save({
            'model_state_dict': self.model.state_dict(),
            'model_optimizer_state_dict': self.model_optim.state_dict(),
            'val_acc': val_acc,
            'validation_year': validation_year,
            'test_acc': test_acc,
            'test_year': test_year,
            'configs': self.configs}, PATH)
    print(f"Model Saved at {PATH}")
    return PATH

  def load_model(self, path):
    checkpoint = torch.load(path, map_location=torch.device(self.device))
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.model_optim.load_state_dict(checkpoint['model_optimizer_state_dict'])
    print(f"Model loaded from {path}")
    print(f"Model Validation Accuracy: {checkpoint['val_acc']*100:.2f}% | Test Accuracy: {checkpoint['test_acc']*100:.2f}%")
    """try:
      self.configs = checkpoint['configs']
    except:
      print("No Configs Found in torch file")"""
    return

## Simple Positional Embeds

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023

    ## Training
    run_name: str = "scale_card_run2"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)

In [ ]:
trainer.run()

Training model: scale_card_run1



  0%|          | 0/21 [00:00<?, ?it/s]

Year: 2000 | Validation Accuracy: 48.81% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 0.00%
Year: 2001 | Validation Accuracy: 49.19% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 0.00%
Training Years: 1990~1999 | Training Accuracy: 50.16% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 49.00% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 50.79% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_scale_card_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Year: 2000 | Validation Accuracy: 51.19% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 49.00%
Year: 2001 | Validation Accuracy: 52.42% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 49.00%
Training Years: 1990~1999 | Training Accuracy: 51.62% | Training OL Accuracy: 53.

  5%|▍         | 1/21 [03:10<1:03:29, 190.49s/it]

Training Years: 1990~1999 | Training Accuracy: 68.24% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 51.80% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 55.80%
Year: 2002 | Test Accuracy: 53.57% | Only Long Accuracy: 44.44%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 55.80% | Highest Test Accuracy in 2002: 53.57%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_scale_card_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 55.80% | Test Accuracy: 51.98%


...

Training Years: 2009~2018 | Training Accuracy: 85.69% | Training OL Accuracy: 54.45%
Year: 2019~2020 Validation Accuracy: 49.70% | Only Long Accuracy: 58.42% | Highest Validation Accuracy: 53.27%
Year: 2021 | Test Accuracy: 54.37% | Only Long Accuracy: 56.75%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 53.27% | Highest Test Accuracy in 2021: 54.37%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_scale_card_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 53.27% | Test Accuracy: 51.98%


<ipython-input-18-8d1d36013527>:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2020 | Validation Accuracy: 48.22% | Only Long Accuracy: 57.31% | Highest Validation Accuracy: 0.00%
Year: 2021 | Validation Accuracy: 52.38% | Only Long Accuracy: 56.75% | Highest Validation Accuracy: 0.00%
Training Years: 2010~2019 | Training Accuracy: 83.82% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 50.30% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 51.79% | Only Long Accuracy: 43.03%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_scale_card_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Year: 2020 | Validation Accuracy: 54.15% | Only Long Accuracy: 57.31% | Highest Validation Accuracy: 50.30%
Year: 2021 | Validation Accuracy: 52.38% | Only Long Accuracy: 56.75% | Highest Validation Accuracy: 50.30%
Training Years: 2010~2019 | Training Accuracy: 83.47% | Training OL Accuracy: 54.

100%|██████████| 21/21 [1:10:17<00:00, 200.83s/it]

Training Years: 2010~2019 | Training Accuracy: 87.68% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 53.07% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 55.45%
Year: 2022 | Test Accuracy: 53.78% | Only Long Accuracy: 43.03%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 55.45% | Highest Test Accuracy in 2022: 53.78%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2000~2001 : 55.80% | Test Accuracy in 2002: 51.98%
Validation Accuracy in 2001~2002 : 56.00% | Test Accuracy in 2003: 50.40%
Validation Accuracy in 2002~2003 : 57.14% | Test Accuracy in 2004: 53.17%
Validation Accuracy in 2003~2004 : 54.96% | Test Accuracy in 2005: 51.98%
Validation Accuracy in 2004~2005 : 56.75% | Test Accuracy in 2006: 51.79%
Validation Accuracy in 2005~2

## Onehot Time Embeds

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = True
    datetime_features: int = 55

    ## Training
    run_name: str = "onehottime_card_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Data Normalization: True
DateTime is one-hot: True
Directory Already Exists at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_onehottime_card_run1


In [ ]:
trainer.run()

Training model: onehottime_card_run1



  0%|          | 0/21 [00:00<?, ?it/s]

Year: 2000 | Validation Accuracy: 46.83% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 0.00%
Year: 2001 | Validation Accuracy: 47.58% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 0.00%
Training Years: 1990~1999 | Training Accuracy: 53.09% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 47.20% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 44.44% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_onehottime_card_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Year: 2000 | Validation Accuracy: 48.81% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 47.20%
Year: 2001 | Validation Accuracy: 49.60% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 47.20%
Training Years: 1990~1999 | Training Accuracy: 50.79% | Training OL Accuracy

  5%|▍         | 1/21 [03:13<1:04:25, 193.29s/it]

Training Years: 1990~1999 | Training Accuracy: 74.56% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 49.20% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 54.00%
Year: 2002 | Test Accuracy: 48.81% | Only Long Accuracy: 44.44%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 54.00% | Highest Test Accuracy in 2002: 48.81%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_onehottime_card_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 54.00% | Test Accuracy: 51.19%


...

Training Years: 2009~2018 | Training Accuracy: 89.59% | Training OL Accuracy: 54.45%
Year: 2019~2020 Validation Accuracy: 47.92% | Only Long Accuracy: 58.42% | Highest Validation Accuracy: 52.67%
Year: 2021 | Test Accuracy: 47.22% | Only Long Accuracy: 56.75%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 52.67% | Highest Test Accuracy in 2021: 47.22%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_onehottime_card_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 52.67% | Test Accuracy: 48.02%


<ipython-input-42-910957380e4c>:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2020 | Validation Accuracy: 50.99% | Only Long Accuracy: 57.31% | Highest Validation Accuracy: 0.00%
Year: 2021 | Validation Accuracy: 44.05% | Only Long Accuracy: 56.75% | Highest Validation Accuracy: 0.00%
Training Years: 2010~2019 | Training Accuracy: 83.07% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 47.52% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 47.01% | Only Long Accuracy: 43.03%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_reset:False_onehottime_card_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Year: 2020 | Validation Accuracy: 50.20% | Only Long Accuracy: 57.31% | Highest Validation Accuracy: 47.52%
Year: 2021 | Validation Accuracy: 45.63% | Only Long Accuracy: 56.75% | Highest Validation Accuracy: 47.52%
Training Years: 2010~2019 | Training Accuracy: 86.09% | Training OL Accuracy

100%|██████████| 21/21 [1:11:21<00:00, 203.86s/it]

Training Years: 2010~2019 | Training Accuracy: 91.69% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 49.31% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 50.89%
Year: 2022 | Test Accuracy: 49.40% | Only Long Accuracy: 43.03%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 50.89% | Highest Test Accuracy in 2022: 49.40%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2000~2001 : 54.00% | Test Accuracy in 2002: 51.19%
Validation Accuracy in 2001~2002 : 53.00% | Test Accuracy in 2003: 50.79%
Validation Accuracy in 2002~2003 : 53.37% | Test Accuracy in 2004: 51.19%
Validation Accuracy in 2003~2004 : 55.56% | Test Accuracy in 2005: 47.22%
Validation Accuracy in 2004~2005 : 54.56% | Test Accuracy in 2006: 51.79%
Validation Accuracy in 2005~2

## Dow30 Pretrained

### Pretraining on Dow30

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = False
    datetime_features: int = 55

    ## Training
    run_name: str = "5yr_dow30_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "amzn axp amgn aapl ba cat csco cvx gs hd hon ibm intc jnj ko jpm mcd mmm mrk msft nke pg trv unh crm vz v wmt dis"
    rolling_window: int = 5 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['amzn', 'axp', 'amgn', 'aapl', 'ba', 'cat', 'csco', 'cvx', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'ko', 'jpm', 'mcd', 'mmm', 'mrk', 'msft', 'nke', 'pg', 'trv', 'unh', 'crm', 'vz', 'v', 'wmt', 'dis']

Dataset Start Year: 2010 | End Year: 2023
years: [2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
 2023]
Data Normalization: True
DateTime is one-hot: False
Save File Directory Made at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1



In [ ]:
trainer.run()

Training model: 5yr_dow30_run1



  0%|          | 0/6 [00:00<?, ?it/s]

Year: 2015 | Validation Accuracy: 50.97% | Only Long Accuracy: 49.79% | Highest Validation Accuracy: 0.00%
Year: 2016 | Validation Accuracy: 52.91% | Only Long Accuracy: 52.26% | Highest Validation Accuracy: 0.00%
Training Years: 2010~2014 | Training Accuracy: 54.04% | Training OL Accuracy: 52.38%
Year: 2015~2016 Validation Accuracy: 51.94% | Only Long Accuracy: 51.03% | Highest Validation Accuracy: 0.00%
Year: 2017 | Test Accuracy: 53.59% | Only Long Accuracy: 54.47%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2015, 2016]_testYear:2017.pt
Year: 2015 | Validation Accuracy: 51.42% | Only Long Accuracy: 49.79% | Highest Validation Accuracy: 51.94%
Year: 2016 | Validation Accuracy: 52.74% | Only Long Accuracy: 52.26% | Highest V

 17%|█▋        | 1/6 [50:32<4:12:42, 3032.49s/it]

Training Years: 2010~2014 | Training Accuracy: 73.72% | Training OL Accuracy: 52.38%
Year: 2015~2016 Validation Accuracy: 63.41% | Only Long Accuracy: 51.03% | Highest Validation Accuracy: 64.70%
Year: 2017 | Test Accuracy: 58.95% | Only Long Accuracy: 54.47%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2015~2016: 64.70% | Highest Test Accuracy in 2017: 58.95%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2015, 2016]_testYear:2017.pt
Model Validation Accuracy: 64.70% | Test Accuracy: 62.85%


...

Training Years: 2014~2018 | Training Accuracy: 79.46% | Training OL Accuracy: 52.35%
Year: 2019~2020 Validation Accuracy: 69.88% | Only Long Accuracy: 53.36% | Highest Validation Accuracy: 71.09%
Year: 2021 | Test Accuracy: 70.59% | Only Long Accuracy: 52.24%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 71.09% | Highest Test Accuracy in 2021: 70.59%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 71.09% | Test Accuracy: 71.83%


<ipython-input-42-910957380e4c>:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2020 | Validation Accuracy: 69.33% | Only Long Accuracy: 51.38% | Highest Validation Accuracy: 0.00%
Year: 2021 | Validation Accuracy: 69.95% | Only Long Accuracy: 52.24% | Highest Validation Accuracy: 0.00%
Training Years: 2015~2019 | Training Accuracy: 76.82% | Training OL Accuracy: 52.84%
Year: 2020~2021 Validation Accuracy: 69.64% | Only Long Accuracy: 51.81% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 66.77% | Only Long Accuracy: 49.51%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Year: 2020 | Validation Accuracy: 69.51% | Only Long Accuracy: 51.38% | Highest Validation Accuracy: 69.64%
Year: 2021 | Validation Accuracy: 70.31% | Only Long Accuracy: 52.24% | Highest V

100%|██████████| 6/6 [5:19:38<00:00, 3196.45s/it]

Training Years: 2015~2019 | Training Accuracy: 80.04% | Training OL Accuracy: 52.84%
Year: 2020~2021 Validation Accuracy: 69.37% | Only Long Accuracy: 51.81% | Highest Validation Accuracy: 70.95%
Year: 2022 | Test Accuracy: 67.62% | Only Long Accuracy: 49.51%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 70.95% | Highest Test Accuracy in 2022: 67.62%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2015~2016 : 64.70% | Test Accuracy in 2017: 62.85%
Validation Accuracy in 2016~2017 : 67.38% | Test Accuracy in 2018: 68.95%
Validation Accuracy in 2017~2018 : 69.12% | Test Accuracy in 2019: 69.92%
Validation Accuracy in 2018~2019 : 71.65% | Test Accuracy in 2020: 71.32%
Validation Accuracy in 2019~2020 : 71.09% | Test Accuracy in 2021: 71.83%
Validation Accuracy in 2020~2

### FineTuning

In [63]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'short_term_forecast'
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = False
    datetime_features: int = 55

    ## Training
    run_name: str = "nvda_run2"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "nvda"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [64]:
configs = Args()
trainer = Trainer(configs)
path = "/content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt"
trainer.load_model(path)

Loading following tickers: ['nvda']

Dataset Start Year: 2001 | End Year: 2023
years: [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]
Data Normalization: True
DateTime is one-hot: False
Directory Already Exists at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[nvda]_valYrs:2_testYrs1_reset:False_nvda_run2
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_5yr_dow30_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Model Validation Accuracy: 70.95% | Test Accuracy: 68.37%


<ipython-input-62-26738e4bc0c3>:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


In [65]:
trainer.run()

Training model: nvda_run2



  0%|          | 0/10 [00:00<?, ?it/s]

Year: 2011 | Validation Accuracy: 67.06% | Only Long Accuracy: 45.24% | Highest Validation Accuracy: 0.00%
Year: 2012 | Validation Accuracy: 69.20% | Only Long Accuracy: 47.60% | Highest Validation Accuracy: 0.00%
Training Years: 2001~2010 | Training Accuracy: 71.93% | Training OL Accuracy: 50.70%
Year: 2011~2012 Validation Accuracy: 68.13% | Only Long Accuracy: 46.41% | Highest Validation Accuracy: 0.00%
Year: 2013 | Test Accuracy: 69.44% | Only Long Accuracy: 50.40%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[nvda]_valYrs:2_testYrs1_reset:False_nvda_run2/reset:False_valYear:[2011, 2012]_testYear:2013.pt
Year: 2011 | Validation Accuracy: 65.87% | Only Long Accuracy: 45.24% | Highest Validation Accuracy: 68.13%
Year: 2012 | Validation Accuracy: 68.00% | Only Long Accuracy: 47.60% | Highest Validation Accuracy: 68.13%
Training Years: 2001~2010 | Training Accuracy: 73.72% | Training OL Accuracy: 50.70%
Ye

 10%|█         | 1/10 [03:24<30:39, 204.40s/it]

Training Years: 2001~2010 | Training Accuracy: 87.79% | Training OL Accuracy: 50.70%
Year: 2011~2012 Validation Accuracy: 67.93% | Only Long Accuracy: 46.41% | Highest Validation Accuracy: 70.32%
Year: 2013 | Test Accuracy: 63.89% | Only Long Accuracy: 50.40%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2011~2012: 70.32% | Highest Test Accuracy in 2013: 63.89%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[nvda]_valYrs:2_testYrs1_reset:False_nvda_run2/reset:False_valYear:[2011, 2012]_testYear:2013.pt
Model Validation Accuracy: 70.32% | Test Accuracy: 68.25%


...

Training Years: 2009~2018 | Training Accuracy: 90.86% | Training OL Accuracy: 51.99%
Year: 2019~2020 Validation Accuracy: 70.89% | Only Long Accuracy: 56.63% | Highest Validation Accuracy: 74.26%
Year: 2021 | Test Accuracy: 72.62% | Only Long Accuracy: 53.57%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 74.26% | Highest Test Accuracy in 2021: 72.62%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[nvda]_valYrs:2_testYrs1_reset:False_nvda_run2/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 74.26% | Test Accuracy: 70.63%


<ipython-input-62-26738e4bc0c3>:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2020 | Validation Accuracy: 69.96% | Only Long Accuracy: 58.50% | Highest Validation Accuracy: 0.00%
Year: 2021 | Validation Accuracy: 71.03% | Only Long Accuracy: 53.57% | Highest Validation Accuracy: 0.00%
Training Years: 2010~2019 | Training Accuracy: 86.41% | Training OL Accuracy: 51.95%
Year: 2020~2021 Validation Accuracy: 70.50% | Only Long Accuracy: 56.04% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 69.32% | Only Long Accuracy: 48.61%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[nvda]_valYrs:2_testYrs1_reset:False_nvda_run2/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Year: 2020 | Validation Accuracy: 71.94% | Only Long Accuracy: 58.50% | Highest Validation Accuracy: 70.50%
Year: 2021 | Validation Accuracy: 72.62% | Only Long Accuracy: 53.57% | Highest Validation Accuracy: 70.50%
Training Years: 2010~2019 | Training Accuracy: 84.26% | Training OL Accuracy: 51.95%
Ye

100%|██████████| 10/10 [34:19<00:00, 205.95s/it]

Training Years: 2010~2019 | Training Accuracy: 90.46% | Training OL Accuracy: 51.95%
Year: 2020~2021 Validation Accuracy: 67.92% | Only Long Accuracy: 56.04% | Highest Validation Accuracy: 72.28%
Year: 2022 | Test Accuracy: 69.32% | Only Long Accuracy: 48.61%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 72.28% | Highest Test Accuracy in 2022: 69.32%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2011~2012 : 70.32% | Test Accuracy in 2013: 68.25%
Validation Accuracy in 2012~2013 : 71.12% | Test Accuracy in 2014: 71.83%
Validation Accuracy in 2013~2014 : 71.03% | Test Accuracy in 2015: 66.67%
Validation Accuracy in 2014~2015 : 70.24% | Test Accuracy in 2016: 65.87%
Validation Accuracy in 2015~2016 : 69.64% | Test Accuracy in 2017: 66.93%
Validation Accuracy in 2016~2